# Hospital Facilities Insight 

Data obtained from Community Benefits Insight: http://www.communitybenefitinsight.org/api/get_hospitals.php

## Import Libaries

In [1]:
# import appropriate packages
import os
import requests
import json 
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import pymysql.cursors
from pymysql import IntegrityError

import warnings
warnings.filterwarnings("ignore") #supress warnings

## Import Data

In [2]:
# API endpoint
api_url = 'http://www.communitybenefitinsight.org/api/get_hospitals.php'

# requesting data from API
response = requests.get(api_url)

# check status
if response.status_code == 200:
    data = response.json()

    # data update check
    last_modified_header = response.headers.get('Last-Modified')

    if last_modified_header:
        last_modified = datetime.strptime(last_modified_header, '%a, %d %b %Y %H:%M:%S GMT')
        print(f"Data was last modified: {last_modified}")

    # convert data to dataframe in chunks
    chunk_size = 100  # define size of chunk
    hospital_dfs = []  # create list to store df 

    for chunk_start in range(0, len(data), chunk_size):
        chunk_end = min(chunk_start + chunk_size, len(data))
        chunk_data = data[chunk_start:chunk_end]
        hospital_df = pd.DataFrame(chunk_data)
        hospital_dfs.append(hospital_df)

    # concate all df chunks into one
    hospital_df = pd.concat(hospital_dfs, ignore_index=True)

    # print df 
    print(hospital_df)
else:
    print(f"Failed to fetch data. Status code: {response.status_code}")
    print(response.text)

     hospital_id hospital_org_id        ein  \
0              1               1  630307951   
1              2               2  630578923   
2              3               3  630312913   
3              4               4  630459034   
4              5               5  581973570   
...          ...             ...        ...   
3486        3487            2647  813040663   
3487        3488            2304  741109643   
3488        3489            2648  831954982   
3489        3490            2302  750800661   
3490        3491            2649  831869297   

                                     name  \
0                Mizell Memorial Hospital   
1                        St Vincents East   
2           Shelby Baptist Medical Center   
3            Callahan Eye Foundation Hosp   
4                 Cherokee Medical Center   
...                                   ...   
3486          Bsw Medical Center - Austin   
3487              Ascension Seton Bastrop   
3488         Texas Health Hosp

In [3]:
hospital_df # print df 

,hospital_id,hospital_org_id,ein,name,name_cr,street_address,city,state,zip_code,fips_state_and_county_code,hospital_bed_count,chrch_affl_f,urban_location_f,children_hospital_f,memb_counc_teach_hosps_f,medicare_provider_number,county,hospital_bed_size,updated_dt
0,1,1,630307951,Mizell Memorial Hospital,Mizell Memorial Hospital,702 Main Street,Opp,AL,36462,01039,99,N,N,N,N,010007,Covington County,<100 beds,"November 20, 2023"
1,2,2,630578923,St Vincents East,St Vincents East,50 Medical Park Drive East,Birmingham,AL,35235,01073,362,N,Y,N,Y,010011,Jefferson County,>299 beds,"November 20, 2023"
2,3,3,630312913,Shelby Baptist Medical Center,Shelby Baptist Medical Center,1000 First Street North,Alabaster,AL,35007,01117,252,N,Y,N,N,010016,Shelby County,100-299 beds,"November 20, 2023"
3,4,4,630459034,Callahan Eye Foundation Hosp,Callahan Eye Foundation Hosp,1720 University Boulevard,Birmingham,AL,35233,01073,106,N,Y,N,Y,010018,Jefferson County,100-299 beds,"November 20, 2023"
4,5,5,581973570,Cherokee Medical Center,Cherokee Medical Center,400 Northwood Drive,Centre,AL,35960,01019,60,N,N,N,N,010022,Cherokee County,<100 beds,"November 20, 2023"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3486,3487,2647,813040663,Bsw Medical Center - Austin,Bsw Medical Center - Austin,5245 W Us 290,Austin,TX,78735,48453,16,N,Y,N,N,670136,Travis County,<100 beds,"November 20, 2023"
3487,3488,2304,741109643,Ascension Seton Bastrop,Ascension Seton Bastrop,630 Highway 71 W,Bastrop,TX,78602,48021,7,N,Y,N,N,670143,Bastrop County,<100 beds,"November 20, 2023"
3488,3489,2648,831954982,Texas Health Hospital Frisco,Texas Health Hospital Frisco,12400 N Dallas Parkway,Frisco,TX,75033,48121,63,N,Y,N,N,670260,Denton County,<100 beds,"November 20, 2023"
3489,3490,2302,750800661,Methodist Midlothian Medical Center,Methodist Midlothian Medical Center,1201 E Highway 287,Midlothian,TX,76065,48139,46,N,Y,N,N,670300,Ellis County,<100 beds,"November 20, 2023"


In [4]:
# print 10 random samples from the dataframe 
hospital_df.sample(10)

,hospital_id,hospital_org_id,ein,name,name_cr,street_address,city,state,zip_code,fips_state_and_county_code,hospital_bed_count,chrch_affl_f,urban_location_f,children_hospital_f,memb_counc_teach_hosps_f,medicare_provider_number,county,hospital_bed_size,updated_dt
3310,3311,2528,390837206,Lakeview Medical Center Of Rice Lake,Lakeview Medical Center Of Rice Lake,1700 West Stout Street,Rice Lake,WI,54868,55005,75,N,N,N,N,520011,Barron County,<100 beds,"November 20, 2023"
171,172,125,814791043,Queen Of The Valley Medical Center,Queen Of The Valley Medical Center,1000 Trancas Street,Napa,CA,94558,06055,171,Y,Y,N,Y,050009,Napa County,100-299 beds,"November 20, 2023"
2860,2861,2249,680673354,Roane County Medical Center,Roane County Medical Center,8045 Roane Medical Center Drive,Harriman,TN,37748,47145,105,N,Y,N,N,440031,Roane County,100-299 beds,"November 20, 2023"
708,709,506,990251372,Molokai General Hospital,Molokai General Hospital,280 Home Olu,Kaunakakai,HI,96748,15009,13,N,Y,N,N,121303,Maui County,<100 beds,"November 20, 2023"
2531,2532,2004,930591528,Legacy Mount Hood Medical Center,Legacy Mount Hood Medical Center,24800 Se Stark Street,Gresham,OR,97030,41051,115,N,Y,N,N,380025,Multnomah County,100-299 beds,"November 20, 2023"
356,357,227,680273974,Stanislaus Surgical,Stanislaus Surgical,1421 Oakdale Road,Modesto,CA,95355,06099,23,N,Y,N,N,050726,Stanislaus County,<100 beds,"November 20, 2023"
1780,1781,1383,810241755,Cabinet Peaks Medical Center,Cabinet Peaks Medical Center,209 Health Park Drive,Libby,MT,59923,30053,25,N,N,N,N,271320,Lincoln County,<100 beds,"November 20, 2023"
1948,1949,1532,221487166,East Orange General Hospital,,300 Central Ave,East Orange,NJ,07018,34013,211,N,Y,N,N,310083,Essex County,100-299 beds,"November 20, 2023"
1069,1070,789,481172106,Rock Regional Hospital,Rock Regional Hospital,3251 N Rock Rd,Derby,KS,67037,20173,31,N,Y,N,N,170204,Sedgwick County,<100 beds,"November 20, 2023"
96,97,62,860983474,Southeast Arizona Medical Center,Southeast Arizona Medical Center,2174 West Oak Avenue,Douglas,AZ,85607,04003,25,N,Y,N,N,031303,Cochise County,<100 beds,"November 20, 2023"


## Data Inspection

In [5]:
hospital_df.shape 

(3491, 19)

In [6]:
hospital_df.info() #all variables in df are labeled as object variables

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3491 entries, 0 to 3490
Data columns (total 19 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   hospital_id                 3491 non-null   object
 1   hospital_org_id             3491 non-null   object
 2   ein                         3491 non-null   object
 3   name                        3491 non-null   object
 4   name_cr                     3491 non-null   object
 5   street_address              3491 non-null   object
 6   city                        3491 non-null   object
 7   state                       3491 non-null   object
 8   zip_code                    3491 non-null   object
 9   fips_state_and_county_code  3491 non-null   object
 10  hospital_bed_count          3491 non-null   object
 11  chrch_affl_f                3491 non-null   object
 12  urban_location_f            3491 non-null   object
 13  children_hospital_f         3491 non-null   obje

In [7]:
columns_to_convert = ['hospital_bed_count', 'medicare_provider_number']
binary_columns = ['chrch_affl_f', 'urban_location_f', 'children_hospital_f', 'memb_counc_teach_hosps_f']

# iterate through the list of columns
for col in columns_to_convert:
    # convert to numeric, coerce errors to NaN
    hospital_df[col] = pd.to_numeric(hospital_df[col], errors='coerce')
    
for col in binary_columns:
    # Map 'N' to 0 and 'Y' to 1
    hospital_df[col] = hospital_df[col].map({'N': 0, 'Y': 1})

# convert updated_dt to date type
hospital_df['updated_dt'] = hospital_df['updated_dt'].apply(lambda x: datetime.strptime(x, '%B %d, %Y').strftime('%Y-%m-%d'))

# check the data types after conversion
hospital_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3491 entries, 0 to 3490
Data columns (total 19 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   hospital_id                 3491 non-null   object
 1   hospital_org_id             3491 non-null   object
 2   ein                         3491 non-null   object
 3   name                        3491 non-null   object
 4   name_cr                     3491 non-null   object
 5   street_address              3491 non-null   object
 6   city                        3491 non-null   object
 7   state                       3491 non-null   object
 8   zip_code                    3491 non-null   object
 9   fips_state_and_county_code  3491 non-null   object
 10  hospital_bed_count          3491 non-null   int64 
 11  chrch_affl_f                3491 non-null   int64 
 12  urban_location_f            3491 non-null   int64 
 13  children_hospital_f         3491 non-null   int6

In [30]:
hospital_df.nunique() #unique variables 

hospital_id                   3491
hospital_org_id               2377
ein                           2377
name                          3330
name_cr                       3072
street_address                3439
city                          2192
state                           51
zip_code                      3200
fips_state_and_county_code    1751
hospital_bed_count             696
chrch_affl_f                     2
urban_location_f                 2
children_hospital_f              2
memb_counc_teach_hosps_f         2
medicare_provider_number      3491
county                        1195
hospital_bed_size                3
updated_dt                       1
dtype: int64

In [8]:
hospital_df.isnull().sum() #null values 

hospital_id                   0
hospital_org_id               0
ein                           0
name                          0
name_cr                       0
street_address                0
city                          0
state                         0
zip_code                      0
fips_state_and_county_code    0
hospital_bed_count            0
chrch_affl_f                  0
urban_location_f              0
children_hospital_f           0
memb_counc_teach_hosps_f      0
medicare_provider_number      0
county                        1
hospital_bed_size             0
updated_dt                    0
dtype: int64

In [9]:
duplicate_rows = hospital_df[hospital_df.duplicated()]

print("Duplicate Rows except first occurrence:")
print(duplicate_rows)

if hospital_df.duplicated().any():
    print("Duplicates exist in the DataFrame.")
else:
    print("No duplicates found in the DataFrame.")

Duplicate Rows except first occurrence:
Empty DataFrame
Columns: [hospital_id, hospital_org_id, ein, name, name_cr, street_address, city, state, zip_code, fips_state_and_county_code, hospital_bed_count, chrch_affl_f, urban_location_f, children_hospital_f, memb_counc_teach_hosps_f, medicare_provider_number, county, hospital_bed_size, updated_dt]
Index: []
No duplicates found in the DataFrame.


## Table Manipulation

In [10]:
# Hospital Information Table

# create unique ID for address
hospital_df['address_id'] = hospital_df['street_address'] + '_' + hospital_df['city'] + '_' + hospital_df['state'] + '_' + hospital_df['zip_code'] + '_' + hospital_df['fips_state_and_county_code']
hospital_df['address_id'] = hospital_df['address_id'].rank(method='dense', ascending=False).astype(int)

#create unique ID for hospital capacity
hospital_df['capacity_id'] = hospital_df['hospital_id'].astype(str) + '_' + hospital_df['hospital_bed_count'].astype(str) + '_' + hospital_df['hospital_bed_size']
hospital_df['capacity_id'] = hospital_df['capacity_id'].rank(method='dense', ascending=False).astype(int)


# create hospital information table
hospital = hospital_df[['hospital_id', 'hospital_org_id', 'name', 'ein', 'medicare_provider_number', 'address_id', 'capacity_id', 'updated_dt']]
hospital

,hospital_id,hospital_org_id,name,ein,medicare_provider_number,address_id,capacity_id,updated_dt
0,1,1,Mizell Memorial Hospital,630307951,10007,609,2381,2023-11-20
1,2,2,St Vincents East,630578923,10011,1075,1270,2023-11-20
2,3,3,Shelby Baptist Medical Center,630312913,10016,3323,667,2023-11-20
3,4,4,Callahan Eye Foundation Hosp,630459034,10018,2440,556,2023-11-20
4,5,5,Cherokee Medical Center,581973570,10022,1337,445,2023-11-20
...,...,...,...,...,...,...,...,...
3486,3487,2647,Bsw Medical Center - Austin,813040663,670136,949,730,2023-11-20
3487,3488,2304,Ascension Seton Bastrop,741109643,670143,718,729,2023-11-20
3488,3489,2648,Texas Health Hospital Frisco,831954982,670260,2901,728,2023-11-20
3489,3490,2302,Methodist Midlothian Medical Center,750800661,670300,2967,726,2023-11-20


In [11]:
# Church Affiliation Table
church_affiliation = hospital_df[['hospital_id', 'chrch_affl_f', 'updated_dt']]
church_affiliation

,hospital_id,chrch_affl_f,updated_dt
0,1,0,2023-11-20
1,2,0,2023-11-20
2,3,0,2023-11-20
3,4,0,2023-11-20
4,5,0,2023-11-20
...,...,...,...
3486,3487,0,2023-11-20
3487,3488,0,2023-11-20
3488,3489,0,2023-11-20
3489,3490,0,2023-11-20


In [12]:
# Hospital Capacity Table extracting data from hospital_df
hospital_capacity = hospital_df[['capacity_id', 'hospital_bed_size', 'hospital_bed_count', 'updated_dt']]
hospital_capacity

,capacity_id,hospital_bed_size,hospital_bed_count,updated_dt
0,2381,<100 beds,99,2023-11-20
1,1270,>299 beds,362,2023-11-20
2,667,100-299 beds,252,2023-11-20
3,556,100-299 beds,106,2023-11-20
4,445,<100 beds,60,2023-11-20
...,...,...,...,...
3486,730,<100 beds,16,2023-11-20
3487,729,<100 beds,7,2023-11-20
3488,728,<100 beds,63,2023-11-20
3489,726,<100 beds,46,2023-11-20


In [13]:
# Children Hospital Table - extracting data from hospital_df
children_hospital= hospital_df[['hospital_id', 'children_hospital_f', 'updated_dt']]
children_hospital

,hospital_id,children_hospital_f,updated_dt
0,1,0,2023-11-20
1,2,0,2023-11-20
2,3,0,2023-11-20
3,4,0,2023-11-20
4,5,0,2023-11-20
...,...,...,...
3486,3487,0,2023-11-20
3487,3488,0,2023-11-20
3488,3489,0,2023-11-20
3489,3490,0,2023-11-20


In [14]:
# Teaching Table - extracting data from hospital_df
teaching = hospital_df[['hospital_id', 'memb_counc_teach_hosps_f', 'updated_dt']]
teaching

,hospital_id,memb_counc_teach_hosps_f,updated_dt
0,1,0,2023-11-20
1,2,1,2023-11-20
2,3,0,2023-11-20
3,4,1,2023-11-20
4,5,0,2023-11-20
...,...,...,...
3486,3487,0,2023-11-20
3487,3488,0,2023-11-20
3488,3489,0,2023-11-20
3489,3490,0,2023-11-20


In [16]:
# Urban Table
urban = hospital_df[['urban_location_f', 'updated_dt','hospital_id']]
urban

,urban_location_f,updated_dt,hospital_id
0,0,2023-11-20,1
1,1,2023-11-20,2
2,1,2023-11-20,3
3,1,2023-11-20,4
4,0,2023-11-20,5
...,...,...,...
3486,1,2023-11-20,3487
3487,1,2023-11-20,3488
3488,1,2023-11-20,3489
3489,1,2023-11-20,3490


In [17]:
# connect to database

username = "jeremiahf24"
password = "$ads507password"
hostName = "ads507finalproject.mysql.database.azure.com"

conn = pymysql.connect(host=hostName,
                       port=3306,
                       user=username,
                       passwd=password, 
                       db= "ads507final", 
                       ssl={"fake_flag_to_enable_tls": True})

In [18]:
tableNames=pd.read_sql_query("""SHOW TABLES""", conn)

# print table names 
tableNames

,Tables_in_ads507final
0,address_location
1,children_hospital
2,church_affiliation
3,hospital
4,hospital_capacity
5,teaching
6,urban


In [20]:
# hospital table - convert df to list of tuples
hospital_data = [tuple(x) for x in hospital.values]

# SQL query - insert data into hospital table
sql_insert_hospital = """
INSERT INTO hospital (hospital_id, hospital_org_id, name, ein, medicare_provider_number, Address_id, capacity_id, updated_dt)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
"""

# check if insertion is needed
if hospital_data:
    # execute query 
    cursor = conn.cursor()
    try:
        cursor.executemany(sql_insert_hospital, hospital_data)
        conn.commit()
    except IntegrityError as e:
        print("Error", e)
        conn.rollback()  # rollback the transaction if an error 
    finally:
        cursor.close()
else:
    print("No new data to insert into hospital table.")

Error (1452, 'Cannot add or update a child row: a foreign key constraint fails (`ads507final`.`hospital`, CONSTRAINT `hospital_address_location_Address_id_fk` FOREIGN KEY (`Address_id`) REFERENCES `address_location` (`Address_id`))')


In [21]:
# children hospital table - convert df to list of tuples
children_hospital_data = [tuple(x) for x in children_hospital.values]

# SQL query - insert data into children_hospital table
sql_insert_children_hospital = """
INSERT INTO children_hospital(hospital_id, children_hospital_f, updated_dt)
VALUES (%s, %s, %s)
"""

# check if insertion is needed
if children_hospital_data:
    # execute query
    cursor = conn.cursor()
    try:
        cursor.executemany(sql_insert_children_hospital, children_hospital_data)
        conn.commit()
    except IntegrityError as e:
        print("Error", e)
        conn.rollback()  # rollback the transaction if an error occurs 
    finally:
        cursor.close()
else:
    print("No new data to insert into children_hospital table.")

Error (1452, 'Cannot add or update a child row: a foreign key constraint fails (`ads507final`.`children_hospital`, CONSTRAINT `children_hospital_hospital_hospital_id_fk` FOREIGN KEY (`hospital_id`) REFERENCES `hospital` (`hospital_id`))')


In [22]:
# hospital_capacity table - convert df to list of tuples
hospital_capacity_data = [tuple(x) for x in hospital_capacity.values]

# SQL query - insert data into hospital_capacity table
sql_insert_hospital_capacity = """
INSERT INTO hospital_capacity (capacity_id, hospital_bed_size, hospital_bed_count, updated_dt)
VALUES (%s, %s, %s, %s)
"""

# check if insertion is needed
if hospital_capacity_data:
    # execute query 
    cursor = conn.cursor()
    try:
        cursor.executemany(sql_insert_hospital_capacity, hospital_capacity_data)
        conn.commit()
    except IntegrityError as e:
        print("Error", e)
        conn.rollback()  # rollback the transaction if an error 
    finally:
        cursor.close()
else:
    print("No new data to insert into hospital_capacity table.")

In [23]:
# teaching table - convert df to list of tuples
teaching_data = [tuple(x) for x in teaching.values]

# SQL query - insert data into children_hospital table
sql_insert_teaching = """
INSERT INTO teaching (hospital_id, memb_counc_teach_hosps_f, updated_dt)
VALUES (%s, %s, %s)
"""

# execute query 
if teaching_data:
    # Execute query 
    cursor = conn.cursor()
    try:
        cursor.executemany(sql_insert_teaching, teaching_data)
        conn.commit()
    except IntegrityError as e:
        print("Error", e)
        conn.rollback()  # rollback the transaction if an error occurs 
    finally:
        cursor.close()
else:
    print("No new data to insert into teaching table.")

Error (1452, 'Cannot add or update a child row: a foreign key constraint fails (`ads507final`.`teaching`, CONSTRAINT `teaching_hospital_hospital_id_fk` FOREIGN KEY (`hospital_id`) REFERENCES `hospital` (`hospital_id`))')


In [24]:
# church_affiliation table - convert df to list of tuples
church_affiliation_data = [tuple(x) for x in church_affiliation.values]

# SQL query - insert data into church_affiliation table
sql_insert_church_affiliation = """
INSERT INTO church_affiliation (hospital_id, chrch_affl_f, updated_dt)
VALUES (%s, %s, %s)
"""

# execute query 
if church_affiliation_data:
    # Execute query 
    cursor = conn.cursor()
    try:
        cursor.executemany(sql_insert_church_affiliation, church_affiliation_data)
        conn.commit()
    except IntegrityError as e:
        print("Error", e)
        conn.rollback()  # rollback the transaction if an error occurs 
    finally:
        cursor.close()
else:
    print("No new data to insert into church_affiliation table.")

In [25]:
# Create a cursor
cursor = conn.cursor()

# Execute SQL queries to get table names
cursor.execute("SHOW TABLES")

# Fetch all the tables
tables = cursor.fetchall()

for table in tables:
        table_name = table[0]

        # Execute SQL query to fetch sample data from each table
        cursor.execute(f"SELECT * FROM {table_name} LIMIT 5")

        # Fetch and print sample data
        sample_data = cursor.fetchall()
        print(f"Sample data from table '{table_name}':")
        for row in sample_data:
            print(row)

Sample data from table 'address_location':
Sample data from table 'children_hospital':
Sample data from table 'church_affiliation':
(1, 1, b'0', datetime.date(2023, 11, 20))
(2, 2, b'0', datetime.date(2023, 11, 20))
(3, 3, b'0', datetime.date(2023, 11, 20))
(4, 4, b'0', datetime.date(2023, 11, 20))
(5, 5, b'0', datetime.date(2023, 11, 20))
Sample data from table 'hospital':
Sample data from table 'hospital_capacity':
(1, '<100 beds', 69, datetime.date(2023, 11, 20), None)
(2, '<100 beds', 25, datetime.date(2023, 11, 20), None)
(3, '100-299 beds', 234, datetime.date(2023, 11, 20), None)
(4, '100-299 beds', 150, datetime.date(2023, 11, 20), None)
(5, '>299 beds', 649, datetime.date(2023, 11, 20), None)
Sample data from table 'teaching':
Sample data from table 'urban':
